# Case Study

## Setup

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from termcolor import colored, cprint
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing
from sklearn.cluster import DBSCAN

In [21]:
#Help funcs for colored output
def green(txt):
    return f"\x1b[32m{txt}\x1b[0m"
def red(txt):
    return f"\x1b[31m{txt}\x1b[0m"
def blue(txt):
    return f"\x1b[36m{txt}\x1b[0m"
def bold(txt):
        return f"\x1b[1m{txt}\x1b[0m"

In [22]:
#Get raw Data
df = pd.read_csv("data/csv/house_data_training.csv", sep=';') 
# remove unnamed column
df = df.iloc[:, 1:]
#Transform string to datetime
df["date"] = pd.to_datetime(df["date"])
df.tail()
#Drop rows with NaN values

maeList = []#for model selection

In Task 01 we have identified 3 lines with missing values. Due to the size of the data set, we can use the listwise Deletion - method at this point.

In [23]:
def dropMissingValues(df):
    df = df.dropna()
    df = df.reset_index(drop=True)
    return df
print(df.index)
df = dropMissingValues(df)   
print(df.index)

RangeIndex(start=0, stop=15000, step=1)
RangeIndex(start=0, stop=14997, step=1)


## Modeling
In Task 01_Exploration various qualitative problems within the data were identified. In this chapter different approaches are implemented to solve these problems.

In [24]:
#Saving a copy of the original df
df_raw = df.copy()

### A) Basic Regression Model
> Creating a basic linear Regression

In [25]:
def reg_train_test(X_train, X_test, y_train, y_test):
    '''Function for building Basic Regression Model'''

    # fit the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # evaluate the model
    ypred = model.predict(X_test)
    
    # evaluate predictions
    mae = mean_absolute_error(y_test, ypred)
    maeList.append(np.round(mae))
    #print(f'{bold("Mean Absolute Error")}: {blue(np.round(mae))}\n')

    print(bold(f'MAE_List expanded:'))
    for i, m in enumerate(maeList):
        if i+1 == len(maeList):
            print(f'model_{bold(i)} - "Mean Absolute Error:" {blue(m)}\nScore: {np.round(score, 4)}') 
        else:
            print(f'model_{bold(i)} - "Mean Absolute Error:" {m}\nScore: {np.round(score, 4)}')
 
    return model

Creating the baseline regression model with the data barely edited (only missing values filtered)

In [26]:
def splitData(df, test_size = 0.2):
    '''function for splitting the data from a given df into the given test_size proportions'''
    from sklearn.model_selection import train_test_split
    # Select price as label and remove price_data from list
    X, y = df.drop(columns=["price"]), df["price"]
    # Transform Column to a numeric value
    if 'date' in df:
        X[["date"]] = X[["date"]].apply(pd.to_numeric)
    # Dataframes in numpy-Arrays konvertieren
    X,y  = np.array(X.values.tolist()), np.array(y.values.tolist())
    #split Data and train the model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state=1)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = splitData(df_raw, 0.2)
model_0 = reg_train_test(X_train, X_test, y_train, y_test)

MAE_List expanded:
model_0 - "Mean Absolute Error:" 6328488.0
Score: 0.0


### B) Data Preprocessing

> Detect Outliers by building 3 Models


In [27]:
#to remove the detected outliers from out trainingData(X_train and y_train) 
#we want to transform the traingsData from np_arrays to dfs and reverse for better handling

def np_to_df(numpy_arr, column_list):
    df = pd.DataFrame(numpy_arr, columns=column_list)
    return df

def df_to_np(df):
    np_arr = df.to_numpy()
    return np_arr

def calc_model_after_drop(df, outlier_index_list):
    '''function creates and evaluates the model after deleting the outliers from TRAINING_DATA given from the parameter outlier_index_list'''
    #create columnList to transform X_Train
    column_list = df.columns.to_list().remove('price')

    X_train, X_test, y_train, y_test = splitData(df, 0.2)

    #transfrom train Data into df to drop the outliers
    df_X_Train = np_to_df(X_train, column_list)
    df_y_Train = np_to_df(y_train, ['price'])

    #calculate max index --> we only want to delete the outliers below this threshold
    maxIndex = df_X_Train.index.stop

    for o in list(outlier_index_list):
        #rint(type(o))
        if o >= maxIndex:
            outlier_index_list.remove(o) 
                 
    #drop the outlierts from the dfs 
    df_X_Train = df_X_Train.drop(df_X_Train.index[outlier_index_list])
    df_y_Train = df_y_Train.drop(df_y_Train.index[outlier_index_list])

    #transfrom back trainigdata to np_arrays
    X_train = df_to_np(df_X_Train)
    y_train = df_to_np(df_y_Train)

    #evaluate model after dropping Outliers from training data
    model = reg_train_test(X_train, X_test, y_train, y_test)

    return model

#### Data quality issues

In Task1 we detected multiple rows with the value 9999999 in its price column which can be considered as Noise values. With the following function we can drop those.

In [28]:
#get thte indexes for the detected price values which are way too high.
def get99(df=df_raw):
    list99 = df.index[df['price'] == 9999999.9].tolist()
    list90 = df.index[df['price'] == 99999999.0].tolist()
    list99_combined =  list(set(list99) | set(list90))
    return list99_combined

> Darf man 99.9 werte aus testdaten droppen??? ansonsten traingsdaten mit mittelwert zu ersetzten

In [29]:
#Create and evaluate model after dropping the 99... values
outlier_list_99 = get99(df_raw)
model_1 = calc_model_after_drop(df_raw, outlier_list_99)

MAE_List expanded:
model_0 - "Mean Absolute Error:" 6328488.0
Score: 0.0
model_1 - "Mean Absolute Error:" 6276384.0
Score: 0.0


In [30]:
def mean99(df=df_raw):
    pass

> after dropping the 9999999 values we get a much lower mae

In [31]:
def drop99_all(df, outlier_index_list):
    return df.drop(df.index[outlier_index_list])

In [32]:
outlier_list_99 = get99(df_raw)
df = drop99_all(df_raw,outlier_list_99)
column_list = df.columns.to_list().remove('price')
X_train, X_test, y_train, y_test = splitData(df, 0.2)
model_0 = reg_train_test(X_train, X_test, y_train, y_test)


MAE_List expanded:
model_0 - "Mean Absolute Error:" 6328488.0
Score: 1.0
model_1 - "Mean Absolute Error:" 6276384.0
Score: 1.0
model_2 - "Mean Absolute Error:" 166876.0
Score: 1.0


##### Outliers
In this figure we can see the distribution of the values for the different features. Some histograms show a skewed distribution. Sometimes you can immediately recognize Outliers.

Method 1:  based on descriptive statistics (Univariate outlier handling)

In [33]:
def z_score(df=df_raw, std_multiply=3):
    '''Univariate outlier detection based on descriptive statistics (three standard deviations)
    can be useful to identify extreme outliers'''

    feature_list=['price', 'bedrooms', 'bathrooms', 'sqft_living',
        'sqft_lot', 'floors', 'dis_super', 'view', 'condition',
        'grade', 'sqft_above', 'sqft_basement',
        'sqft_living15', 'sqft_lot15']

    outliers_dict = {}#dict for storing outlierts for an outlier summary df
    outlier_list_unique = []
    print(bold("Potential Outliers:"))
    for feature in feature_list:
        feature_data = df[feature]

        df_feature = pd.concat([feature_data], axis=1)
        df_feature["outlier"] = 0

        three_std=feature_data.std()*std_multiply
        mean=feature_data.mean()

        inlier_low=mean-three_std
        inlier_high=mean+three_std

        outlier_list = [] #list for storing indexes of outliers
        for i, value in enumerate(feature_data):
            if value < inlier_low or value > inlier_high:
                outlier_list.append(i)
                df_feature.iloc[i,1] = 1      

        print(f'{bold(feature)} detected: {blue(len(outlier_list))}')
        
        if not len(outlier_list) == 0:
            outliers_dict[str(feature)]=outlier_list
            outlier_list_unique =  list(set(outlier_list_unique) | set(outlier_list))
    
    return outlier_list_unique

In [34]:
#get indexes of outlier Rows 
outlier_list_z_score = z_score(df_raw, 4)
model_2 = calc_model_after_drop(df_raw, outlier_list_z_score)

Potential Outliers:
price detected: 421
bedrooms detected: 42
bathrooms detected: 26
sqft_living detected: 55
sqft_lot detected: 202
floors detected: 0
dis_super detected: 0
view detected: 210
condition detected: 0
grade detected: 10
sqft_above detected: 47
sqft_basement detected: 42
sqft_living15 detected: 33
sqft_lot15 detected: 179
MAE_List expanded:
model_0 - "Mean Absolute Error:" 6328488.0
Score: 0.0
model_1 - "Mean Absolute Error:" 6276384.0
Score: 0.0
model_2 - "Mean Absolute Error:" 166876.0
Score: 0.0
model_3 - "Mean Absolute Error:" 6255913.0
Score: 0.0


> principal components als methode für 2 dimensionale dargstellung geeignet (nicht sicher ob es in outlierhandling passt)

Method 2: bsed on distances (Multivariate outlier handling)

In [35]:
def outliers_knn(df=df_raw, k=3, num_outliers=181):
    #X_train needed
    X_train, X_test, y_train, y_test = splitData(df, 0.2)

    #normalize data to identify outliers
    scaler = preprocessing.MinMaxScaler()
    X = scaler.fit_transform(X_train)

    nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree').fit(X)
    distances, indices = nbrs.kneighbors(X)

    outlier_indices=np.argpartition(distances[:,1],-num_outliers)[-num_outliers:]
 
    return outlier_indices

In [36]:
#get indexes of outlier Rows 
outlier_list_knn = outliers_knn(df_raw, 5, 1000) #181
model_3 = calc_model_after_drop(df_raw, outlier_list_knn)

MAE_List expanded:
model_0 - "Mean Absolute Error:" 6328488.0
Score: 0.0
model_1 - "Mean Absolute Error:" 6276384.0
Score: 0.0
model_2 - "Mean Absolute Error:" 166876.0
Score: 0.0
model_3 - "Mean Absolute Error:" 6255913.0
Score: 0.0
model_4 - "Mean Absolute Error:" 6488984.0
Score: 0.0


Method 3: based on density clustering (Multivariate outlier handling)

In [37]:
def outliers_dbscan(df=df_raw, k=3, num_outliers=181, eps=0.42, min_samples=5):

    #need distances
    X_train, X_test, y_train, y_test = splitData(df, 0.2)
    scaler = preprocessing.MinMaxScaler()
    X = scaler.fit_transform(X_train)
    nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='ball_tree').fit(X)
    distances, indices = nbrs.kneighbors(X)

    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(X)

    inliers_list=[]
    outliers_list=[]
    index_upper=distances[:,1].size

    for index in range (0,index_upper):
        if clustering.labels_[index] == -1:
            outliers_list.append(index)
        else:
            inliers_list.append(index)

    return outliers_list

In [38]:
#get indexes of outlier Rows 
outlier_list_dbscan = outliers_dbscan()
model_4 = calc_model_after_drop(df_raw, outlier_list_dbscan)

MAE_List expanded:
model_0 - "Mean Absolute Error:" 6328488.0
Score: 0.0
model_1 - "Mean Absolute Error:" 6276384.0
Score: 0.0
model_2 - "Mean Absolute Error:" 166876.0
Score: 0.0
model_3 - "Mean Absolute Error:" 6255913.0
Score: 0.0
model_4 - "Mean Absolute Error:" 6488984.0
Score: 0.0
model_5 - "Mean Absolute Error:" 6727487.0
Score: 0.0


##### Conclusion Outliers Handling

In [39]:
baseline = maeList[0]
for i, model in enumerate(maeList):
    if model > baseline:
        print(red(f'model: {i}: {model}'))
    elif model == baseline:
        print(f'model: {i}: {model}')
    else:
        print(green(f'model: {i}: {model}'))

model: 0: 6328488.0
model: 1: 6276384.0
model: 2: 166876.0
model: 3: 6255913.0
model: 4: 6488984.0
model: 5: 6727487.0


##### Noise

> 999.9 Werte?

In [40]:
#Split DataSet into data and target
def getNoise(df, cv=5):

    df_noise = df.drop(['date'], axis = 1)
    x = df_noise.iloc[:,2:]
    y = df_noise.iloc[:,1]

    #Regressions Modelle
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.linear_model import BayesianRidge
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import VotingRegressor
    from sklearn.model_selection import cross_val_predict
    from sklearn.metrics import mean_absolute_error

    reg1 = GradientBoostingRegressor(random_state=1)
    reg2 = BayesianRidge()
    reg3 = DecisionTreeRegressor(max_depth=5, random_state=1)

    reg1.fit(x,y)
    reg2.fit(x,y)
    reg3.fit(x,y)

    ereg = VotingRegressor([('gb', reg1),('brr',  reg2),('dtr', reg3)])

    ereg.fit(x, y)

    y_pred=cross_val_predict(ereg,x,y, cv=cv)

    xt = x[:20]
    #real = y[:20]
    pred1 = reg1.predict(xt)
    pred2 = reg2.predict(xt)
    pred3 = reg3.predict(xt)
    pred4 = ereg.predict(xt)
    y_pred20 = y_pred[:20]

    mae=mean_absolute_error(y_pred,y)
    noise_id=[]
    for index,i in enumerate(y):
        if y_pred[index] > i+mae*10:
            noise_id.append(index)
        elif y_pred[index] < i-mae*10:
            noise_id.append(index)    

    print(f"Bei dem 10fachen MAE kann man bis zu {red(len(noise_id))} Noise Sätze finden")
    noise_index_list = df_noise.index.tolist()
    return noise_index_list

In [41]:
noise_index_list = getNoise(df_raw)
model = calc_model_after_drop(df_raw, outlier_list_z_score)
model

Bei dem 10fachen MAE kann man bis zu 427 Noise Sätze finden
MAE_List expanded:
model_0 - "Mean Absolute Error:" 6328488.0
Score: 0.0
model_1 - "Mean Absolute Error:" 6276384.0
Score: 0.0
model_2 - "Mean Absolute Error:" 166876.0
Score: 0.0
model_3 - "Mean Absolute Error:" 6255913.0
Score: 0.0
model_4 - "Mean Absolute Error:" 6488984.0
Score: 0.0
model_5 - "Mean Absolute Error:" 6727487.0
Score: 0.0
model_6 - "Mean Absolute Error:" 6255913.0
Score: 0.0


LinearRegression()

##### Transformed, standardized or normalized

#### Data Reduction issues

##### Feature Selection / Instance Selection

In [42]:
#get list of features which show a higher corrleation with the label
def getRelFeatures(df=df_raw):
    corr =df.corr(method="spearman")
    rel_features =[]
    corr_fig = corr["price"]
    ix = corr.sort_values('price', ascending=False).index
    print(bold("Relevante Korrelationen:"))
    for i in ix:
        if corr_fig[i]>= 0.3 or corr_fig[i]<=-0.3:
            rel_features.append(i)
            print("Corr", bold(i),"zum Label:", green(round(corr_fig[i],3)))
        else:
            print("Corr", bold(i),"zum Label:", red(round(corr_fig[i],3)))
            
    return rel_features

In [43]:
def drop_features(df, feature_list):
    return df[feature_list]

In [44]:
rel_features = getRelFeatures()
df = drop_features(df_raw, rel_features)
column_list = df.columns.to_list().remove('price')
X_train, X_test, y_train, y_test = splitData(df, 0.2)
model = reg_train_test(X_train, X_test, y_train, y_test)


Relevante Korrelationen:
Corr price zum Label: 1.0
Corr grade zum Label: 0.64
Corr sqft_living zum Label: 0.606
Corr sqft_living15 zum Label: 0.542
Corr sqft_above zum Label: 0.516
Corr bathrooms zum Label: 0.493
Corr lat zum Label: 0.459
Corr floors zum Label: 0.346
Corr bedrooms zum Label: 0.318
Corr view zum Label: 0.268
Corr sqft_basement zum Label: 0.229
Corr yr_built zum Label: 0.167
Corr waterfront zum Label: 0.099
Corr yr_renovated zum Label: 0.083
Corr long zum Label: 0.079
Corr zipcode zum Label: 0.034
Corr sqft_lot zum Label: 0.015
Corr sqft_lot15 zum Label: 0.009
Corr id zum Label: 0.008
Corr dis_super zum Label: 0.006
Corr ahf1 zum Label: 0.005
Corr ahf3 zum Label: 0.002
Corr ahf2 zum Label: -0.005
Corr condition zum Label: -0.019
MAE_List expanded:
model_0 - "Mean Absolute Error:" 6328488.0
Score: 0.0
model_1 - "Mean Absolute Error:" 6276384.0
Score: 0.0
model_2 - "Mean Absolute Error:" 166876.0
Score: 0.0
model_3 - "Mean Absolute Error:" 6255913.0
Score: 0.0
model_4 - "M

> hinweise aus der vorlesung die interessant sein könnten:
* principal components
* EFA – Determine number of factors

#### Algorithm Selection: Experiment with different regression algorithms, e.g. linear regression, polynomial regression, regression trees etc.


pass

####  Hyper-parameter Tuning: Change the hyper-parameters of your algorithms (e.g.„degree“ in case of polynomial regression)

pass

## Modeling Continued

## Evaluation